## Get phenotypes associated with a list of genes


### Set up 


In [7]:
import json
#from pprint import pprint

In [8]:
import requests

INDEX_PAGE = "http://idr-demo.openmicroscopy.org/webclient/?experimenter=-1"

# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

### First read in the table with the list of genes
The example contains 
* a gene that has an image annotated with a phenotype not mapped to any ontology (PSMA1)
* a gene that has no images annotated with any phenotypes (PSMA5)
* a gene that images that are annotated with multiple phenotypes and some phenotypes are mapped to more than one ontology term (PREX2)
* a gene that is not annotated to any images (NOTAGENE)


In [9]:
with open('FiveExampleGenes.txt') as f:
    genes = f.read().splitlines()

### Check that the table has been read in properly - print out first 5 values and check the length of the list

In [10]:
genes[:5]


['PSMA1', 'PSMA2', 'PSMA5', 'PREX2', 'NOTAGENE']

In [11]:
len (genes)

5

### Set up URLS we are going to use to query IDR

In [12]:
SCREENS_PROJECTS_URL = "http://idr-demo.openmicroscopy.org/mapr/api/{key}/?value={value}"
PLATES_URL = "http://idr-demo.openmicroscopy.org/mapr/api/{key}/plates/?value={value}&id={screen_id}"
IMAGES_URL = "http://idr-demo.openmicroscopy.org/mapr/api/{key}/images/?value={value}&node={parent_type}&id={parent_id}"
ATTRIBUTES_URL = "http://idr-demo.openmicroscopy.org/webclient/api/annotations/?type=map&image={image_id}"

### Then get the phenotypes

For each gene in our list
* get which screens its in
* get the plates in the screens with the gene in
* then the images
* then the annotations of those images

NOTE: to be added - should also do this for datasets

#### First open up the file that the results will be written to and create the column header row

In [13]:
file = open('GenesWithPhenotypes.txt','w') 
file.write('Gene Symbol\tScreen\tPlate\tImageID\tAuthorPhenotype\tCMPOTerm\tCMPOAccession\n')

#### Then iterate through each gene in the list and fetch the phenotypes

In [18]:


for gene in genes:
    #file.write ('{0}'.format(gene))
    qs1 = {'key': 'gene', 'value': gene}
    url1 = SCREENS_PROJECTS_URL.format(**qs1)
    
    for s in session.get(url1).json()['screens']:
        screen_id = s['id']
        screen_name = s['name']
        #print (gene, s['id'], s['name'])    
        qs2 = {'key': 'gene', 'value': gene, 'screen_id': screen_id}
        url2 = PLATES_URL.format(**qs2)
        
        for p in session.get(url2).json()['plates']:
            plate_id = p['id'] 
            plate_name = p['name']
            #print (gene, p['id'], p['name'])  
            qs3 = {'key': 'gene', 'value': gene, 'parent_type': 'plate', 'parent_id': plate_id}
            url3 = IMAGES_URL.format(**qs3)
            
            # now to get the phenotypes linked to the images - not outputting this correctly yet
            
            # gene - might be not associated with any images
            # gene - associated with image but none with phenotypes
            # gene - image with one phenotype but no ontology mapping
            # gene - image with one phenotype plus one ontology mapping
            # gene - image with one phenotype plus multiple ontology mappings (elongated plus protrustions)
            # gene - image with multiple phenotypes with mixture of number of ontology mappings
            # 
            # best structure for this is dictionary of dictionaries?
            
            
            
            
            for i in session.get(url3).json()['images']:

                # create a list in which to create each row of genes and their phenotypes
                geneRow = []
                
                
                image_id = str(i['id'])
                url4 = ATTRIBUTES_URL.format(**{'image_id': image_id})
                for a in session.get(url4).json()['annotations']:
                    outRow = {}
                    outRow['gene'] = gene
                    outRow['screen'] = screen_name
                    outRow['plate'] = plate_name
                    outRow['image_id'] = image_id
                    print "Values", a['values']
                    
                    for kvpair in a['values']:
                        #print(a['values'])
                        #print "kv", kvpair
                        
                        if 'Phenotype' in kvpair:
                            #print (kvpair[1])
                            #geneRow.extend([gene, screen_name, plate_name, image_id, kvpair[1]])
                            outRow['Phenotype'] = kvpair[1]
                            
                        if 'Phenotype Term Name' in kvpair:
                            #print (kvpair[1])
                            #file.write('\t{0}'.format(kvpair[1])) 
                            #geneRow.append(kvpair[1])
                            outRow['Phenotype Term Name'] = kvpair[1]
                            
                        if 'Phenotype Term Accession' in kvpair:
                            #print (kvpair[1])
                            #file.write('\t{0}'.format(kvpair[1]))
                            #geneRow.append(kvpair[1])
                            outRow['Phenotype Term Accession'] = kvpair[1]
                            
                        #file.write('\n')
             
                   # print outRow
                
            #if len (geneRow) > 0:
            #    #print "row is : ", geneRow
            #    # create a tab-delimted row to print out
            #    separator = '\t'
            #    printRow = separator.join( geneRow )
            #    printRow = printRow + '\n'
            #    print printRow
            #    file.write(printRow)
                
        
file.close()        
    


Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'ORF Identifier', u'5797'], [u'ORF Identifier URL', u'http://horfdb.dfci.harvard.edu/hv5/index.php?page=getresults&by=detail&qury=5797']]
Values [[u'hORFeome V5.1 Clone Description', u'Homo sapiens proteasome (prosome, macropain) subunit, alpha type, 1, transcript variant 2'], [u'ORF Comments', u'hORFeome V5.1 identifier']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Cell Line', u'HeLa'], [u'Channels', u'DAPI:nuclei;TRITC:HA_Flag tagged protein'], [u'Scored As Nuclear Foci At Initial Stage', u'no'], [u'Subcellular Localization Based On Secondary Screening', u'no']]
Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'ORF Identifier', u'5797'], [u'ORF Identifier URL', u'http://horfdb.dfci.harvard.edu/hv5/index.php?page=getresults&by=detail&qury=5

Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'ORF Identifier', u'5797'], [u'ORF Identifier URL', u'http://horfdb.dfci.harvard.edu/hv5/index.php?page=getresults&by=detail&qury=5797']]
Values [[u'hORFeome V5.1 Clone Description', u'Homo sapiens proteasome (prosome, macropain) subunit, alpha type, 1, transcript variant 2'], [u'ORF Comments', u'hORFeome V5.1 identifier']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Cell Line', u'HeLa'], [u'Channels', u'DAPI:nuclei;TRITC:HA_Flag tagged protein'], [u'Scored As Nuclear Foci At Initial Stage', u'no'], [u'Subcellular Localization Based On Secondary Screening', u'no']]
Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'ORF Identifier', u'442'], [u'ORF Identifier URL', u'http://horfdb.dfci.harvard.edu/hv5/index.php?page=getresults&by=detail&qury=44

Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'ORF Identifier', u'442'], [u'ORF Identifier URL', u'http://horfdb.dfci.harvard.edu/hv5/index.php?page=getresults&by=detail&qury=442']]
Values [[u'hORFeome V5.1 Clone Description', u'Homo sapiens proteasome (prosome, macropain) subunit, alpha type, 1, transcript variant 2'], [u'ORF Comments', u'hORFeome V5.1 identifier']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Cell Line', u'HeLa'], [u'Channels', u'DAPI:nuclei;TRITC:HA_Flag tagged protein'], [u'Scored As Nuclear Foci At Initial Stage', u'no'], [u'Subcellular Localization Based On Secondary Screening', u'no']]
Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'ORF Identifier', u'442'], [u'ORF Identifier URL', u'http://horfdb.dfci.harvard.edu/hv5/index.php?page=getresults&by=detail&qury=442'

Values [[u'Cell Line', u'HeLa'], [u'Quality Control', u'pass'], [u'Channels', u'dapi: DNA;vsvg-cfp: CFP-tsO45G ;pm-647: cell surface tsO45G']]
Values [[u'Gene Identifier', u'ENSG00000129084'], [u'Gene Identifier URL', u'http://www.ensembl.org/id/ENSG00000129084'], [u'Gene Symbol', u'PSMA1']]
Values [[u'Gene Annotation Comments', u'Ensembl release 81 gene symbol added by IDR curator'], [u'Analysis Gene Annotation Build', u'GRCh37, Ensembl release 61, Feb 2011, Gene Symbols from Ensembl release 61 or 81']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'siRNA Identifier', u'38346'], [u'siRNA Pool Identifier', u'']]
Values [[u'Sense Sequence', u'GGGCAGGAUUCAUCAAAUUTT'], [u'Antisense Sequence', u'AAUUUGAUGAAUCCUGCCCTG'], [u'Reagent Design Gene Annotation Build', u'NCBI35, Ensembl release 27, Dec 2004']]
Values [[u'Cell Line', u'HeLa'], [u'Quality Control', u'pass'], [u'Channels', u'dapi: DNA;vsvg-cfp: CFP-tsO45G ;pm-647: cell surface tsO45G']]
Values [[u'Gene Identifier', u'ENSG00000129

Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/5682'], [u'Gene Symbol', u'PSMA1']]
Values [[u'RefSeq Accession', u'NM_002786+NM_148976'], [u'RefSeq Hits', u'4+4'], [u'Analysis Gene Annotation Build', u'NCBI36.3, RefSeq release 27, Jan 2008']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Phenotype', u'other phenotype']]
Values [[u'siRNA Identifier', u'D-010123-01'], [u'siRNA Identifier', u'D-010123-03'], [u'siRNA Identifier', u'D-010123-04'], [u'siRNA Identifier', u'D-010123-05'], [u'siRNA Pool Identifier', u'M-010123-01']]
Values [[u'Original GeneID Target', u'5682'], [u'Original Gene Target', u'PSMA1'], [u'Original RefSeq Target', u'NM_002786'], [u'Original LocusLink Target', u'23110933']]
Values [[u'Cell Line', u'HeLa'], [u'Channels', u'Alexa 488:tubulin;Hoechst:DNA;Tritc:actin'], [u'Has Phenotype', u'yes'], [u'Phenotype Annotation Level', u'multiple reagents']]
Values [[u'Gene Identifier', u'5682'], [u'Gene Identifier URL', u

...

Values [[u'Gene Identifier', u'80243'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/80243'], [u'Gene Symbol', u'PREX2']]
Values [[u'Gene Symbol Synonyms', u'DEPDC2'], [u'Gene Annotation Comments', u'Gene symbol updated by IDR curator to be consistent across plates.'], [u'RefSeq Locus', u'NM_025170'], [u'RefSeq GI Number', u'47578116']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Phenotype', u'Large, spread'], [u'Phenotype Term Name', u'increased cell size phenotype'], [u'Phenotype Term Accession', u'CMPO_0000128'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000128']]
Values [[u'Phenotype', u'Spindly'], [u'Phenotype Term Name', u'elongated cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000077'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000077'], [u'Phenotype Term Name', u'cell with projections'], [u'Phenotype Term Accession', u'CMPO_0000071'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo

Values [[u'Gene Identifier', u'80243'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/80243'], [u'Gene Symbol', u'PREX2']]
Values [[u'Gene Symbol Synonyms', u'DEPDC2'], [u'Gene Annotation Comments', u'Gene symbol updated by IDR curator to be consistent across plates.'], [u'RefSeq Locus', u'NM_025170'], [u'RefSeq GI Number', u'47578116']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Phenotype', u'Triangular'], [u'Phenotype Term Name', u'triangular shaped cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000122'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000122'], [u'Phenotype Term Name', u'cell with projections'], [u'Phenotype Term Accession', u'CMPO_0000071'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000071']]
Values [[u'Phenotype', u'Spindly'], [u'Phenotype Term Name', u'elongated cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000077'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo

Values [[u'Gene Identifier', u'80243'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/80243'], [u'Gene Symbol', u'PREX2']]
Values [[u'Gene Symbol Synonyms', u'DEPDC2'], [u'Gene Annotation Comments', u'Gene symbol updated by IDR curator to be consistent across plates.'], [u'RefSeq Locus', u'NM_025170'], [u'RefSeq GI Number', u'47578116']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Phenotype', u'Large, spread'], [u'Phenotype Term Name', u'increased cell size phenotype'], [u'Phenotype Term Accession', u'CMPO_0000128'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000128']]
Values [[u'Phenotype', u'Spindly'], [u'Phenotype Term Name', u'elongated cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000077'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000077'], [u'Phenotype Term Name', u'cell with projections'], [u'Phenotype Term Accession', u'CMPO_0000071'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo

Values [[u'Gene Identifier', u'80243'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/80243'], [u'Gene Symbol', u'PREX2']]
Values [[u'Gene Symbol Synonyms', u'DEPDC2'], [u'Gene Annotation Comments', u'Gene symbol updated by IDR curator to be consistent across plates.'], [u'RefSeq Locus', u'NM_025170'], [u'RefSeq GI Number', u'47578116']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Phenotype', u'Triangular'], [u'Phenotype Term Name', u'triangular shaped cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000122'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000122'], [u'Phenotype Term Name', u'cell with projections'], [u'Phenotype Term Accession', u'CMPO_0000071'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000071']]
Values [[u'Phenotype', u'Spindly'], [u'Phenotype Term Name', u'elongated cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000077'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo

Values [[u'Gene Identifier', u'80243'], [u'Gene Identifier URL', u'http://www.ncbi.nlm.nih.gov/gene/80243'], [u'Gene Symbol', u'PREX2']]
Values [[u'Gene Symbol Synonyms', u'DEPDC2'], [u'Gene Annotation Comments', u'Gene symbol updated by IDR curator to be consistent across plates.'], [u'RefSeq Locus', u'NM_025170'], [u'RefSeq GI Number', u'47578116']]
Values [[u'Organism', u'Homo sapiens']]
Values [[u'Phenotype', u'Triangular'], [u'Phenotype Term Name', u'triangular shaped cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000122'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000122'], [u'Phenotype Term Name', u'cell with projections'], [u'Phenotype Term Accession', u'CMPO_0000071'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo/CMPO_0000071']]
Values [[u'Phenotype', u'Spindly'], [u'Phenotype Term Name', u'elongated cell phenotype'], [u'Phenotype Term Accession', u'CMPO_0000077'], [u'Phenotype Term Accession URL', u'http://www.ebi.ac.uk/cmpo